In [1]:
%%capture
!pip install langchain
!pip install -U langchain-community
!pip install faiss-gpu
!pip install -U bitsandbytes

!pip install accelerate
!pip install chonkie[semantic]


In [ ]:
import re
sep=r"\n\s*NetHack\s+3\.6\s+.*?\d{4}\s*\n+.*?NetHack\s+Guidebook\s+\d+\s*\n+"
file_path = "/root/arushi/Guidebook.txt"  # Replace with the actual file path
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

pattern = r'^\s*NetHack Guidebook\s+\d+\s*$'

pages = [x for x in re.split(pattern, text, flags=re.MULTILINE) if x.strip()]

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# import re

# # Define the regex separator
# separator = r"\n\s*NetHack\s+3\.6\s+.*?\d{4}\s*\n+.*?NetHack\s+Guidebook\s+\d+\s*\n+"

# def split_text_for_rag(text):
#     # Use regex to split text


#     # Initialize RecursiveCharacterTextSplitter
#     splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)

#     # Split into smaller chunks
#     documents = splitter.create_documents(pages)

#     return documents

# # Example usage
# file_path = "/root/arushi/Guidebook.txt"  # Replace with the actual file path
# with open(file_path, "r", encoding="utf-8") as file:
#     text = file.read()

# split_docs = split_text_for_rag(text)
# print(f"Total chunks: {len(split_docs)}")



# from chonkie import SDPMChunker
# from chonkie import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

# # model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

# # Basic initialization with default parameters
# chunker = SDPMChunker(
#     embedding_model=embeddings,
#     threshold=0.5,                              # Similarity threshold (0-1)
#     chunk_size=2048,                             # Maximum tokens per chunk
#     min_sentences=1,                            # Initial sentences per chunk
#     skip_window=1                               # Number of chunks to skip when looking for similarities
# )
# text = "\n".join(pages)
# chunks = chunker.chunk(text)

# for chunk in chunks:
#     print(f"Chunk text: {chunk.text}")
#     print(f"Token count: {chunk.token_count}")
#     print(f"Number of sentences: {len(chunk.sentences)}")

In [ ]:
from chonkie import RecursiveChunker, RecursiveRules

chunker = RecursiveChunker(
    tokenizer="gpt2",
    chunk_size=512,
    rules=RecursiveRules(), # Default rules
    min_characters_per_chunk=12,
)
text = "\n".join(pages)
chunks = chunker.chunk(text)


from langchain_core.documents import Document

pages=[Document(
    page_content=chunk.text,
    metadata={"token_count": chunk.token_count},
) for chunk in chunks]

In [ ]:
for chunk in chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print("===========================/t=============")
    # print(f"Token count: {chunk.token_count}")
    # print(f"Number of sentences: {len(chunk.sentences)}")

Chunk text: 









                           A Guide to the Mazes of Menace
                               (Guidebook for NetHack)


                         Original version - Eric S. Raymond
             (Edited and expanded for 3.6 by Mike Stephenson and others)



          1.  Introduction

               Recently,  you  have  begun to find yourself unfulfilled and
          distant in your daily occupation.  Strange dreams of prospecting,
          stealing,  crusading,  and  combat have haunted you in your sleep
          for many months, but you aren't sure of the reason.   You  wonder
          whether  you have in fact been having those dreams all your life,
          and somehow managed to forget about them until now.  Some  nights
          you awaken suddenly and cry out, terrified at the vivid recollec-
          tion of the strange and powerful creatures that seem to be  lurk-
          ing  behind  every  corner  of  the dungeon in your dream.  Could
          these

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from sentence_transformers import SentenceTransformer

# embedding_model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True, device="cuda" if torch.cuda.is_available() else "cpu")
EMBEDDING_MODEL_NAME = "Snowflake/snowflake-arctic-embed-l"
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    pages, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
import enum

# from .base import NLELanguageWrapper


class Role(enum.Enum):
    ARCHEOLOGIST = "arc"
    BARBARIAN = "bar"
    CAVEMAN = "cav"
    HEALER = "hea"
    KNIGHT = "kni"
    MONK = "mon"
    PRIEST = "pri"
    RANGER = "ran"
    ROGUE = "rog"
    SAMURAI = "sam"
    TOURIST = "tou"
    VALKYRIE = "val"
    WIZARD = "wiz"


ACTIONS = {
    "north": "move north",
    "east": "move east",
    "south": "move south",
    "west": "move west",
    "northeast": "move northeast",
    "southeast": "move southeast",
    "southwest": "move southwest",
    "northwest": "move northwest",
    "far north": "move far north",
    "far east": "move far east",
    "far south": "move far south",
    "far west": "move far west",
    "far northeast": "move far northeast",
    "far southeast": "move far southeast",
    "far southwest": "move far southwest",
    "far northwest": "move far northwest",
    "up": "go up a staircase",
    "down": "go down a staircase (tip: you can only go down if you are standing on the stairs)",
    "wait": "rest one move while doing nothing",
    "more": "display more of the message (tip: ONLY ever use when current message ends with --More--)",
    "annotate": "leave a note about the level",
    "apply": "apply (use) a tool",
    "call": "name a monster or object, or add an annotation",
    "cast": "cast a spell",
    "close": "close an adjacent door",
    "open": "open an adjacent door",
    "dip": "dip an object into something",
    "drop": "drop an item",
    "droptype": "drop specific item types (specify in the next prompt)",
    "eat": "eat something (tip: replenish food when hungry)",
    "esc": "exit menu or message",
    "engrave": "engrave writing on the floor (tip: Elbereth)",
    "enhance": "advance or check weapons skills",
    "fire": "fire ammunition from quiver",
    "fight": "fight a monster (even if you only guess one is there)",
    "force": "force a lock",
    "inventory": "show your inventory",
    "invoke": "invoke ",
    "jump": "jump to a location",
    "kick": "kick an enemy or a locked door or chest",
    "look": "look at what is under you",
    "loot": "loot a box on the floor",
    "monster": "use a monster's special ability (when polymorphed)",
    "offer": "offer a sacrifice to the gods (tip: on an aligned altar)",
    "overview": "display an overview of the dungeon",
    "pay": "pay your shopping bill",
    "pickup": "pick up things at the current location",
    "pray": "pray to the gods for help",
    "puton": "put on an accessory",
    "quaff": "quaff (drink) something",
    "quiver": "select ammunition for quiver",
    "read": "read a scroll or spellbook",
    "remove": "remove an accessory",
    "rub": "rub a lamp or a stone",
    "search": "search for hidden doors and passages",
    "swap": "swap wielded and secondary weapons",
    "takeoff": "take off one piece of armor",
    "takeoffall": "take off all armor",
    "teleport": "teleport to another level (if you have the ability)",
    "throw": "throw something (e.g. a dagger or dart)",
    "travel": "travel to a specific location on the map (tip: in the next action, specify > or < for stairs, { for fountain, and _ for altar)",
    "twoweapon": "toggle two-weapon combat",
    "untrap": "untrap something",
    "wear": "wear a piece of armor",
    "wield": "wield a weapon",
    "wipe": "wipe off your face",
    "zap": "zap a wand",
    "minus": "-",
    "space": " ",
    "apos": "'",
    "0": "0",
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7",
    "8": "8",
    "9": "9",
}


def get_instruction_prompt(task=None):
    action_strings = ",\n".join(f"{action}: {description}" for action, description in ACTIONS.items())
    instruction_prompt = f"""
        You are an agent playing NetHack. The following are the possible actions you can take in the game, followed by a short description of each action:

        {action_strings}.

        Tips:
        - When the message asks for a completion, such as: "What do you want to eat? [d or ?*]", you should respond with a single character corresponding to the item you want to eat/use.
            - For example, "What do you want to eat? [dgh or ?*]" -> Possible answers are "d", "g", or "h" to eat the associated food.
        - When the message asks for a direction, such as: "In what direction?" you should respond with a direction.
        - When the message has --More-- at the end, your next action should be "more" to see the rest of the message.
        - Explore the environment to find the stairs down to the next level.
        - Always carefully read the last message to understand the current state of the game and decide your next action accordingly.
        - If you keep moving in the same direction, you will eventually hit a wall and stop moving. Your message might be: "It's solid stone", or "It's a wall". Change your action to move in another direction to continue exploring the environment.
        - Read the language observation carefully and look at ascii map or image observation provided to decide the next action to take and where to move next.
        - You can attack monsters by moving into them.

        In a moment I will present a history of actions and observations from the game.
        Your goal is to get as far as possible in the game.

        PLAY!
        """.strip()

    return instruction_prompt
print(get_instruction_prompt())

You are an agent playing NetHack. The following are the possible actions you can take in the game, followed by a short description of each action:
        
        north: move north,
east: move east,
south: move south,
west: move west,
northeast: move northeast,
southeast: move southeast,
southwest: move southwest,
northwest: move northwest,
far north: move far north,
far east: move far east,
far south: move far south,
far west: move far west,
far northeast: move far northeast,
far southeast: move far southeast,
far southwest: move far southwest,
far northwest: move far northwest,
up: go up a staircase,
down: go down a staircase (tip: you can only go down if you are standing on the stairs),
wait: rest one move while doing nothing,
more: display more of the message (tip: ONLY ever use when current message ends with --More--),
annotate: leave a note about the level,
apply: apply (use) a tool,
call: name a monster or object, or add an annotation,
cast: cast a spell,
close: close an adjace

In [ ]:
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=50)



Starting retrieval for user_query='How to play nethack?'...


In [ ]:
import torch
from transformers import pipeline

model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

def generate_answer(user_query,n_context=50):


    print(f"\nStarting retrieval for {user_query=}...")
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=n_context)


    context= ''.join([x.page_content.strip() for x in retrieved_docs])

    prompt_in_chat_format = [
        {"role": "system",
            "content": f"{get_instruction_prompt()}"},
        {

            "role": "system",
            "content": """Using the information contained in the context,
    give a comprehensive answer to the question.
    Respond only to the question asked, response should be concise and relevant to the question.
    Provide the number of the source document when relevant.
    If the answer cannot be deduced from the context, do not give an answer.""",
        },
        {
            "role": "user",
            "content": f"""Context:
    {context}
    ---
    Now here is the question you need to answer.

    Question: {user_query}""",
        },
    ]

    return pipe(
        prompt_in_chat_format,
        max_new_tokens=1024,
    )

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
outputs= generate_answer("which is the best character?" ,n_context=50)
print(outputs[0]["generated_text"][-1]['content'])


Starting retrieval for user_query='which is the best character?'...
Based on the information provided, it's not clear who the best character is, as the guidebook doesn't explicitly state a character's name and ranking. However, I can provide some context and insights.

The guidebook mentions that Rangers are most at home in the woods, but some say they're out of place in a dungeon. It also mentions that some nightstalls (i.e., players) have been unfulfilled and distant in their daily occupation, and that some nightstalls have been haunted by strange dreams.

Given these hints, it's possible that the best character is not necessarily a high-ranking or powerful character, but rather one that is agile, quick, and perceptive. This could be an Elf, given their agility, quickness, and perceptive nature.

However, without knowing the character's name and ranking, it's impossible to determine which character is the best. The guidebook provides some context and hints, but it does not explicitl

In [ ]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query="List all the game instructions.", k=50)



In [ ]:
print(''.join([x.page_content.strip() for x in retrieved_docs]))

|    |....|    ----------                                        |
          |    |.<..|####...@...$.|                                        |
          |    |....-#   |...B....+                                        |
          |    |....|    |.d......|                                        |
          |    ------    -------|--                                        |
          |                                                                |
          |                                                                |Alex Kompel           Janne Salmijarvi         Norm Meluch
               Alex Smith         Jean-Christophe Collet      Olaf Seibert
              Andreas Dorn             Jeff Bailey           Pasi Kallinen
               Andy Church            Jochen Erwied            Pat Rankin
              Andy Swanson             John Kallen           Patric Mueller
              Ari Huttunen             John Rupley            Paul Winner
              Barton House             